In [ ]:
 !huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [2]:
!huggingface-cli whoami

joshl1
orgs:  UMBC


## Installations
 Install the necessary libraries.

In [3]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.7 MB/s eta 0:00:00


## Setup the model

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import random

model_name = "meta-llama/Llama-2-7b-chat-hf"

This next step will take a couple minutes to download the model.

In [12]:
# From the Llama-2 Chat Tutorial
# https://colab.research.google.com/drive/1X1z9Q6domMKl2CnEM0QGHNwidLfR4dW2?usp=sharing#scrollTo=THqfvzHIjSK9

quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [16]:
def roll_dice():
    # Simulate a dice roll between 1 and 20
    return random.randint(1, 20)

def generate_game_response(history, action):
    # Combine the game history into a cohesive context
    history_context = " ".join(history[-3:])  # Use the last 3 situations for context

    # Determine success or failure
    dice_result = roll_dice()
    if dice_result >= 10:
        outcome = "success"
    else:
        outcome = "failure"
    #TODO: Critical Success and failure?

    print(f"\n[DICE ROLL: {dice_result}] Your action resulted in a {outcome}.")

    prompt = f"""You are narrating an interactive fiction game. Given the game history and player's action, describe what happens next in 2-3 sentences. The player's action resulted in a {outcome}. Focus only on the immediate new situation.

Game history:
{history_context}

Player action: {action}

What happens next:"""

    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(**model_inputs, max_length=512, temperature=0.7)
    new_situation = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

    # Return only the new situation
    return new_situation

In [17]:
# Generate initial game setting
setting_prompt = """You are creating an interactive fiction game. Describe the initial setting and situation the player finds themselves in. Keep it to 2-3 sentences."""

model_inputs = tokenizer(setting_prompt, return_tensors="pt").to("cuda:0")
output = model.generate(**model_inputs, max_length=1024, temperature=0.7)
initial_situation = tokenizer.decode(output[0], skip_special_tokens=True).strip()

# Initialize game memory with the initial situation
game_memory = [initial_situation]

print("\nWelcome to the AI Generated Adventure!")
print("\nYour situation:", initial_situation)

# Main game loop
while True:
    print("\nWhat would you like to do? (type 'quit' to end)")
    action = input("> ")

    if action.lower() == 'quit':
        print("\nThanks for playing!")
        break

    # Generate response based on memory and player's action
    response = generate_game_response(game_memory, action)
    game_memory.append(response)  # Add the new situation to memory
    print("\n" + response)

    #TODO: Limit the memory size.


Welcome to the AI Generated Adventure!

Your situation: You are creating an interactive fiction game. Describe the initial setting and situation the player finds themselves in. Keep it to 2-3 sentences.

The game begins in a small village nestled in the heart of a dense forest. The player is a young adventurer who has just arrived in the village, seeking fortune and glory in the untamed wilderness beyond. As they explore the village, they notice strange occurrences happening around them, hinting at a dark and sinister force lurking in the shadows.

What would you like to do? (type 'quit' to end)
> I try to sense what is lurking in the shadows.

[DICE ROLL: 19] Your action resulted in a success.

As you focus your senses on the shadows, you pick up on a faint, eerie hum emanating from the nearby woods. The villagers you speak to seem nervous and hesitant to meet your gaze, casting quick glances around before continuing with their daily routines.

What would you like to do? (type 'quit'